In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import scipy

import numpy as np
import pandas as pd
import xarray as xr
import collections
import os 

import warnings
warnings.filterwarnings("ignore")

import xesmf as xe

import dask.distributed as dd
import dask_kubernetes as dk
import dask
import dask.array as da
import rhg_compute_tools.kubernetes as rhgk

from scipy.sparse import coo_matrix

In [2]:
xe.__version__

'0.2.1'

NOTE: Can't replicate Jiawei's example notebook, https://github.com/JiaweiZhuang/sparse_dot/blob/master/scatter_large_weights.ipynb, without getting that pickling error that he claims to have solved in this issue: https://github.com/JiaweiZhuang/xESMF/issues/71. 

### This notebook is a test of all the steps for Spatial Disaggregation to get a handle on the total CPU time it will take for this part of BCSD. 

Once-off steps: 

1. compute multi-decade daily climatologies of ERA-5 at obs-res and coarsen it to model-res (they, e.g. NASA-NEX, do not say how, we will do bilinear for consistency with later step)

Per model/scenario/experiment steps:

1. subtract (or divide for precip) BC’ed model data at model-res from obs climo at model resolution to calculate a “scaling factor” 
2. bilinearly interpolate “scaling factor” (using xESMF) from the model grid to the obs grid 
3. Apply scaling factor by adding (for temp) and multiplying (for precip) the “scaling factor” to the obs-res daily climatology 

NOTE: For the purpose of being conservative with timing, the "coarsen obs climatology step to model-res" is in the per model/scenario/experiment step, since we don't know for sure how/if CMIP6 models will be at exactly the same resolution. 

Currently this workflow is only built out for temperature, not precipitation. All steps are included, the last step (applying the interpolated scale factor to the obs-res daily climatology) has not yet been tested. All other parts of the workflow have been tested. The second to last step, the interpolation of the scaling factor from coarse to fine, is the most memory intensive, thus I have only tested for a subset of timesteps. 

In [3]:
client, cluster = rhgk.get_standard_cluster()
cluster

In [4]:
def _convert_lons(ds, lon_name='longitude'):
    '''
    converts longitudes from 0 to 360 to -180 to 180
    '''
    ds_conv_coords = ds.assign_coords(longitude=(((ds[lon_name] + 180) % 360) - 180))
    ds_sort = ds_conv_coords.sel(**{lon_name: np.sort(ds_conv_coords[lon_name].values)})
    return(ds_sort)

def compute_climo(da):
    climatology = da.groupby('time.dayofyear').mean('time')
    
    return climatology.compute()

Load GMFD (obs) and Berkeley Earth (we are pretending BEST is a climate model that has already undergone bias correction here) 

In [5]:
tmax_obs = xr.open_mfdataset(os.path.join('/gcs/rhg-data/climate/source_data/GMFD/tmax', 
                                         'tmax_0p25_daily_199*'), concat_dim='time',
                              parallel=True).squeeze(drop=True)

In [6]:
# standardize longitudes 
tmax_obs = _convert_lons(tmax_obs)

In [7]:
tmax_model = xr.open_dataset(os.path.join('/gcs/rhg-data/climate/source_data/BEST/TMAX', 
                                         'Complete_TMAX_Daily_LatLong1_1990.nc'))

Remove leap days from model and obs 

In [8]:
# remove leap days 
tmax_obs = tmax_obs.sel(time=~((tmax_obs.time.dt.month == 2) & (tmax_obs.time.dt.day == 29)))
tmax_model = tmax_model.sel(time=~((tmax_model.month == 2) & (tmax_model.day == 29)))

In [9]:
size_model = len(tmax_model.latitude) * len(tmax_model.longitude)
size_obs = len(tmax_obs.latitude) * len(tmax_obs.longitude)

Compute obs climatology (obs res)

Note: in real workflow, this will be pre-computed (only need to do this once) and loaded. 

In [10]:
tmax_obs_future = client.submit(compute_climo, tmax_obs['tmax'])
dd.progress(tmax_obs_future)

VBox()

In [11]:
climo_obs_fine = client.gather(tmax_obs_future)

### Interpolate obs climo: fine -> coarse 

In [12]:
%%time 
obs_to_mod_weights = '/home/jovyan/obs_to_mod_bilinear_weights.nc'
regridder_obs_to_mod = xe.Regridder(tmax_obs.rename({'latitude': 'lat', 'longitude': 'lon'}), tmax_model.rename({'latitude': 'lat', 'longitude': 'lon'}), 
                         'bilinear', filename=obs_to_mod_weights, reuse_weights=True)

Reuse existing file: /home/jovyan/obs_to_mod_bilinear_weights.nc
CPU times: user 24.5 ms, sys: 8.58 ms, total: 33.1 ms
Wall time: 29.8 ms


In [13]:
%%time
climo_obs_coarse = regridder_obs_to_mod(climo_obs_fine)

CPU times: user 721 ms, sys: 205 ms, total: 927 ms
Wall time: 867 ms


### Compute scaling factor by subtracting for temperature, dividing for precip, the BC'ed model data at model-res from obs climo at model-res. 

In [14]:
def calculate_anomaly(ds, climo):
    # Necessary workaround to xarray's check with zero dimensions
    # https://github.com/pydata/xarray/issues/3575
    da = ds['temperature']
    if sum(da.shape) == 0:
        return da
    groupby_type = ds.day_of_year
    gb = da.groupby(groupby_type)
    
    return gb - climo

def compute_scale_factor(spec):
    '''
    computes scale factor at the coarse level 
    '''
    da_adj, da_obs_climo_coarse = spec
    
    return xr.map_blocks(calculate_anomaly, da_adj, args=[da_obs_climo_coarse.rename({'dayofyear': 'day_of_year'})])

In [16]:
# chunk by year 
tmax_model = tmax_model.chunk({'time': 365, 'latitude': len(tmax_model.latitude), 'longitude': len(tmax_model.longitude)})

In [28]:
%%time 
spec = (tmax_model, climo_obs_coarse)
scale_factor_coarse = compute_scale_factor(spec)

CPU times: user 307 ms, sys: 2.6 ms, total: 310 ms
Wall time: 279 ms


In [29]:
scale_factor_coarse['time'] = tmax_model.time.copy()

In [30]:
%%time 
sfc = scale_factor_coarse.persist()

CPU times: user 24.5 ms, sys: 1.03 ms, total: 25.5 ms
Wall time: 24.1 ms


In [32]:
%%time 
sfc = sfc.compute()

'%%time \nsfc = sfc.compute()'

In [33]:
# if starting from this point 
sfc = xr.open_dataset('/home/jovyan/sfc_test.nc')

### Interpolate scaling factor: coarse (model grid) -> fine (obs grid)

In [34]:
%%time
mod_to_obs_weights = '/home/jovyan/mod_to_obs_bilinear_weights.nc'
regridder_mod_to_obs = xe.Regridder(tmax_model.rename({'latitude': 'lat', 'longitude': 'lon'}), 
                                    tmax_obs.rename({'latitude': 'lat', 'longitude': 'lon'}), 
                         'bilinear', filename=mod_to_obs_weights, reuse_weights=True)

Reuse existing file: /home/jovyan/mod_to_obs_bilinear_weights.nc
CPU times: user 69.9 ms, sys: 125 ms, total: 195 ms
Wall time: 183 ms


Functions for replicating xESMF functionality for running on workers since this is too large to run in notebook memory and xESMF is not setup to be used in conjunction with dask. 

In [35]:
def read_xesmf_weights_coo_matrix(weights_file, size_in, size_out):
    ds = xr.open_dataset(weights_file)
    n_s = ds.dims['n_s']
    col = ds['col'].values - 1
    row = ds['row'].values - 1
    S = ds['S'].values
    A = coo_matrix((S, (row, col)), shape=[size_out, size_in]) 
    return A

def apply_weights(spec):
    weights, da, shape_in, shape_out, lats_out, lons_out = spec
    indata = da.values
    
    shape_horiz = shape_in[-2:]
    extra_shape = shape_in[0:-2]
    
    if len(shape_in) > 2:
        indata_flat = indata.reshape(shape_in[0], shape_in[1]*shape_in[2])
    else: 
        indata_flat = indata.reshape(-1, shape_in[0]*shape_in[1])
    
    outdata_flat = weights.dot(indata_flat.T).T
    
    '''if len(shape_in) > 2:
        outdata = outdata_flat.reshape(
            [shape_in[0], shape_out[0], shape_out[1]])
    else: 
        outdata = outdata_flat.reshape(
            [*extra_shape, shape_out[0], shape_out[1]])'''
    outdata = outdata_flat.reshape(
            [shape_out[0], shape_out[1]])
        
    if len(shape_in) > 2:
        dims = {'time': da.time,'latitude': lats_out, 'longitude': lons_out}
        coords = {'time': da.time, 'latitude': lats_out, 'longitude': lons_out}
    else:
        dims = {'latitude': lats_out, 'longitude': lons_out}
        coords = {'latitude': lats_out, 'longitude': lons_out}
    
    
    outdata_da = xr.DataArray(outdata, dims=dims, 
                              coords=coords)
    
    return outdata_da

In [36]:
%%time 
# make scipy sparse weight matrix 
weights_coo_mat = read_xesmf_weights_coo_matrix(mod_to_obs_weights, size_model, size_obs)

CPU times: user 58.8 ms, sys: 25.1 ms, total: 83.9 ms
Wall time: 79.5 ms


Note: the method with `da.map_blocks` only works on a few years, beyond that we run out of memory. But I believe that this is the better way to do it - needs work. 

In [ ]:
'''chunk_size = (1,180,360)
client.rebalance()
spec = (weights_coo_mat, sfc, chunk_size, (720, 1440), tmax_gmfd.latitude, tmax_gmfd.longitude)
sff_lazy = da.map_blocks(apply_weights, spec, dtype=np.float64, chunks=chunk_size)
sff_eager = client.compute(sff_lazy).result()'''

In [ ]:
JOBS = [(weights_coo_mat, sfc['temperature'].isel(time=timestep).drop('time'), (180, 360), (720, 1440), 
         tmax_gmfd.latitude, tmax_gmfd.longitude) for timestep in sfc['temperature'].time]

In [ ]:
sff_futures = client.map(apply_weights, JOBS[:20])

In [ ]:
dd.progress(sff_futures)

In [ ]:
sffs = client.gather(sff_futures)

In [ ]:
sff = xr.concat(sffs, pd.Index(sfc.time.values, name='time'))

### Last step: add (or multiply for precip) the scaling factor to the obs-res daily climatology

NOTE: this step has not been tested given that the memory for the above step needs to be further worked out, but it is essentially the inverse of the step above where we compute the scaling factor. 

In [ ]:
def apply_scale_factor(scale_factor_fine, obs_climo):
    da = ds['temperature']
    if 'dayofyear' in scale_factor_fine.dims:
        scale_factor_fine.rename({'dayofyear': 'day_of_year'})
        
    sff_daily = scale_factor_fine.groupby(scale_factor_fine.day_of_year)
    
    return obs_climo + sff_daily

def apply_scale_factor_wrapper(spec):
    '''
    applies scale factor to obs climatology
    '''
    scale_factor_fine, da_obs_climo_fine = spec
    
    return xr.map_blocks(apply_scale_factor, da_adj, args=[da_obs_climo_fine])

In [ ]:
# chunk by year 
sff_chunk = sff.chunk({'time': 365, 'latitude': len(tmax_obs.latitude), 'longitude': len(tmax_obs.longitude)})

In [ ]:
%%time 
spec = (sff_chunk, climo_obs_fine)
model_ds = apply_scale_factor_wrapper(spec)

In [ ]:
model_ds = model_ds.persist()

In [ ]:
model_downscaled = model_ds.compute()

### Apply standardizing functions for final output and save (probably as zarr array)